## Setup

In [1]:
import sys
from pathlib import Path

from aocd import get_data, submit

current_day is only available in December (EST)


In [2]:
# Add parent directory to path to allow relative imports into Jupyter notebook
sys.path.append(str(Path.cwd().parent))

In [3]:
# Get raw advent-of-code data
data: str = get_data(year=2024, day=15)

## Part a

In [ ]:
from typing import TYPE_CHECKING

if TYPE_CHECKING:
    from collections.abc import Callable
# Imports
from time import sleep

from common.utils.dict_grid import find_object_in_grid, find_objects_in_grid, get_grid_object, text_to_dict, print_grid
from IPython.display import clear_output

In [5]:
# Constants
MOVE_DIRECTIONS: dict[str, complex] = {
    "^": 1j,
    ">": 1,
    "v": -1j,
    "<": -1,
}

In [ ]:
# Functions
def get_direction(move: str) -> complex:
    """Validation of move input."""
    direction = MOVE_DIRECTIONS.get(move)
    if not direction:
        err_msg = f"Move '{move}' is invalid, must be one of '{"','".join(MOVE_DIRECTIONS.keys())}'"
        raise ValueError(err_msg)
    return direction


def move_box(grid: dict[complex, str], position: complex, direction: complex) -> tuple[dict[complex, str], bool]:
    """Try to move box in grid, return new grid and whether the box was moved."""
    grid, box_position = move_object_in_grid(grid, position, direction)
    return grid, box_position == position + direction


def move_object_in_grid(
    grid: dict[complex, str],
    position: complex,
    direction: complex,
    *,
    box_chars: tuple[str, ...] = tuple("O"),  # Box
    move_box_func: Callable[[dict[complex, str], complex, complex], tuple[dict[complex, str], bool]] = move_box,
) -> tuple[dict[complex, str], complex]:
    """Move object and any pushed boxes, return new grid and new position."""
    # Get object to move
    obj = get_grid_object(grid, position, allowed_objects=tuple("@") + box_chars)  # Robot, box

    # Get object at new position
    new_position = position + direction
    obj_at_new_position = get_grid_object(
        grid,
        new_position,
        allowed_objects=tuple(".#") + box_chars,  # Empty space, wall, box
    )

    # Handle empty space
    if obj_at_new_position == ".":
        grid[new_position], grid[position] = obj, "."
        return grid, new_position

    # Handle box collision
    if obj_at_new_position in box_chars:
        # Try to move the box
        grid, box_moved = move_box_func(grid, new_position, direction)

        # If the box was moved, move the object
        if box_moved:
            grid[new_position], grid[position] = obj, "."
            return grid, new_position

    # Otherwise, there is a wall collision, and nothing happens
    return grid, position


def calc_gps(grid: dict[complex, str], box_char: str = "O") -> int:
    """Calculate total GPS coordinates of boxes in a grid."""
    # Find grid height
    grid_height = max(cell.imag for cell in grid)

    # Find boxes in grid
    boxes = find_objects_in_grid(grid, obj=box_char)
    # Calculate GPS coordinates for each box
    return int(sum((grid_height - box.imag) * 100 + box.real for box in boxes))


def perform_move_sequence(
    grid: dict[complex, str],
    moves: list[str],
    *,
    show_grid: bool = False,
    box_chars: tuple[str, ...] = tuple("O"),  # Box
    move_box_func: Callable[[dict[complex, str], complex, complex], tuple[dict[complex, str], bool]] = move_box,
) -> int:
    """Perform a sequence of moves on a robot in the grid and return the GPS coordinates of the boxes."""
    # Find the robot position
    robot_position = find_object_in_grid(grid, obj="@")

    # Move the robot and print the grid every 500 steps
    for i, move in enumerate(moves):
        direction = get_direction(move)
        grid, robot_position = move_object_in_grid(
            grid, robot_position, direction, box_chars=box_chars, move_box_func=move_box_func
        )

        # Print grid every 500 steps
        if show_grid and i % 500 == 0:
            print(f"Move {i}: {move}")
            print_grid(grid)
            sleep(0.1)
            clear_output(wait=True)

    # Calculate GPS coordinates
    return calc_gps(grid, box_char=box_chars[0])

In [7]:
# Split the data into grid and robot moves
grid_str, moves_str = data.split("\n\n")

# Parse the grid and moves
grid = text_to_dict(grid_str)
moves = list(moves_str.replace("\n", ""))

In [10]:
# Perform the move sequence
total_gps_score = perform_move_sequence(grid, moves, show_grid=True)

Move 19500: ^
49  ##################################################
48  #...OOOOO.O....#OOO..#OO...O.O#O..#OO.......OOOOO#
47  ##OOOOOOOOO...O.O#O#.#OO..#......O#OO...........O#
46  #.OOO....#.OO..OOO............#O..OO....#.O......#
45  #OOOO.OO.......O#O.O#O...........OOO..#..O.......#
44  #...O..OO..........O...........OOOO..............#
43  #.O#OOO.O...........O..O.....OOO#...#.......OO#..#
42  #..O...#..........OO.........OOOO......#........O#
41  #O..#OOO......O#....#O...##...OO........O.......O#
40  #.#.OO#O..#...O......#........O.OO.....#O...##.OO#
39  #.OO.#OO...OO..O..OO...OO.....O##......O......OOO#
38  #..OOOO....##......OO.OO......O.......O...O....OO#
37  ##.OOOOO......O.......OO#........OO........O.O..O#
36  #.#..OO......#.....O..###..O.....O#..O..#...#.#..#
35  #.OO..OO.........O...OO#O.....OO#OO.....O....OOOO#
34  #.O#O.#O...O#...#O...#O#O.......OO.........##..O.#
33  ##OO...O...#O...#O...#OO.......##O.........O.O..O#
32  #O.#..#....OOO..OO#..OO#....#..O#..O........OOO

In [ ]:
# Submit answer
submit(total_gps_score, part="a", day=15, year=2024)

## Part b

In [83]:
# Constants
CHARACTER_WIDENING = {
    "#": "##",
    "O": "[]",
    ".": "..",
    "@": "@.",
}

In [ ]:
# Functions
def get_other_side_of_wide_box(grid: dict[complex, str], position: complex) -> tuple[str, complex]:
    """Check if a wide box is valid and return the position and shape of the other side."""
    if grid[position] == "[":
        # Check if box has right side
        if grid[position + 1] != "]":
            err_msg = (
                f"Left side of box at position '{position}' is missing right side, instead found '{grid[position + 1]}'"
            )
            raise ValueError(err_msg)
        return "]", position + 1

    if grid[position] == "]":
        # Check if box has left side
        if grid[position - 1] != "[":
            err_msg = (
                f"Right side of box at position '{position}' is missing left side, instead found '{grid[position - 1]}'"
            )
            raise ValueError(err_msg)
        return "[", position - 1

    err_msg = f"Position '{position}' is not a box side, instead found '{grid[position]}'"
    raise ValueError(err_msg)


def move_wide_box(grid: dict[complex, str], position: complex, direction: complex) -> tuple[dict[complex, str], bool]:
    """Try to move wide box in grid, return new grid and whether the box was moved."""
    # Find the other side of the box
    _other_side, other_side_position = get_other_side_of_wide_box(grid, position)

    # Handle vertical collision
    if direction in (1j, -1j):
        # Try to move both sides of the box
        temp_grid = grid.copy()
        temp_grid, new_position = move_object_in_grid(
            temp_grid, position, direction, box_chars=tuple("[]"), move_box_func=move_wide_box
        )
        temp_grid, new_position_other_side = move_object_in_grid(
            temp_grid, other_side_position, direction, box_chars=tuple("[]"), move_box_func=move_wide_box
        )

        # Only move the box if both sides were moved successfully
        if new_position == position + direction and new_position_other_side == other_side_position + direction:
            # Update grid with new box positions
            grid = temp_grid
            return grid, True

    # Handle horizontal collision
    if direction in (1, -1):
        # Move the other side first
        grid, new_other_side_position = move_object_in_grid(
            grid, other_side_position, direction, box_chars=tuple("[]"), move_box_func=move_wide_box
        )

        # If the other side was moved successfully, move the push-receiving side
        if new_other_side_position == other_side_position + direction:
            grid[position + direction], grid[position] = grid[position], "."
            return grid, True

    # If the box was not moved, return the original grid
    return grid, False

In [180]:
# Create wide version of map
wide_grid_str = grid_str
for original, wide in CHARACTER_WIDENING.items():
    wide_grid_str = wide_grid_str.replace(original, wide)

wide_grid = text_to_dict(wide_grid_str)

In [ ]:
# Perform the move sequence
total_gps_score_wide = perform_move_sequence(
    wide_grid, moves, show_grid=True, box_chars=tuple("[]"), move_box_func=move_wide_box
)

In [ ]:
# Submit answer
submit(total_gps_score_wide, part="b", day=15, year=2024)